In [7]:
import os
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.ensemble import IsolationForest

from xgboost import XGBClassifier
import lightgbm as lgb

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", DEVICE)

X_PATH = "../processed/preprocessed_X_seq.npy"
Y_PATH = "../processed/preprocessed_y_seq.npy"

os.makedirs("results", exist_ok=True)


Using device: cuda


In [8]:
X_seq = np.load(X_PATH)  # shape: (N_windows, 32, 52)
y_seq = np.load(Y_PATH)  # shape: (N_windows, 32)

print("X_seq:", X_seq.shape)
print("y_seq:", y_seq.shape)


X_seq: (78773, 32, 52)
y_seq: (78773, 32)


In [9]:
# Window-level label: 1 if any flow in window is attack, else 0
y_window = (y_seq.max(axis=1) > 0).astype(int)   # shape: (N_windows,)

print("y_window shape:", y_window.shape)
unique, counts = np.unique(y_window, return_counts=True)
print("Window label distribution:", dict(zip(unique, counts)))

# Flatten each window: (32, 52) -> (1664,) for tabular models
N, L, F = X_seq.shape
X_flat = X_seq.reshape(N, L * F)
print("X_flat shape:", X_flat.shape)


y_window shape: (78773,)
Window label distribution: {np.int64(0): np.int64(56230), np.int64(1): np.int64(22543)}
X_flat shape: (78773, 1664)


In [10]:
def split_window_data(X_flat, y_win, train_ratio=0.7, val_ratio=0.15):
    N = X_flat.shape[0]
    n_train = int(N * train_ratio)
    n_val = int(N * val_ratio)
    
    X_train = X_flat[:n_train]
    y_train = y_win[:n_train]
    X_val   = X_flat[n_train:n_train+n_val]
    y_val   = y_win[n_train:n_train+n_val]
    X_test  = X_flat[n_train+n_val:]
    y_test  = y_win[n_train+n_val:]
    return (X_train, y_train), (X_val, y_val), (X_test, y_test)

(X_train, y_train), (X_val, y_val), (X_test, y_test) = split_window_data(X_flat, y_window)

print("Train:", X_train.shape, y_train.shape)
print("Val  :", X_val.shape, y_val.shape)
print("Test :", X_test.shape, y_test.shape)


Train: (55141, 1664) (55141,)
Val  : (11815, 1664) (11815,)
Test : (11817, 1664) (11817,)


In [11]:
def compute_metrics(y_true, y_pred, model_name="Model"):
    acc  = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred, zero_division=0)
    rec  = recall_score(y_true, y_pred, zero_division=0)
    f1   = f1_score(y_true, y_pred, zero_division=0)
    print(f"🔍 {model_name} Metrics:")
    print("Accuracy :", acc)
    print("Precision:", prec)
    print("Recall   :", rec)
    print("F1-score :", f1)
    return acc, prec, rec, f1

# Class weights for supervised models (to help recall)
counts = np.bincount(y_train, minlength=2).astype(float)
weights = counts.sum() / (2 * counts + 1e-8)
print("Class counts:", counts, "-> class weights:", weights)
class_weights_np = weights


Class counts: [43271. 11870.] -> class weights: [0.63715884 2.3227043 ]


In [12]:
xgb = XGBClassifier(
    n_estimators=300,
    max_depth=6,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    tree_method="hist",
    eval_metric="logloss",
    n_jobs=-1,
    random_state=42
)

xgb.fit(X_train, y_train)

y_pred_xgb = xgb.predict(X_test)
acc_xgb, prec_xgb, rec_xgb, f1_xgb = compute_metrics(y_test, y_pred_xgb, "XGBoost")

df_xgb = pd.DataFrame([[ "XGBoost", acc_xgb, prec_xgb, rec_xgb, f1_xgb ]],
                      columns=["Model","Accuracy","Precision","Recall","F1"])
df_xgb.to_csv("results/XGBoost.csv", index=False)
df_xgb


🔍 XGBoost Metrics:
Accuracy : 0.5367690615215368
Precision: 0.9906976744186047
Recall   : 0.03746701846965699
F1-score : 0.07220338983050847


,Model,Accuracy,Precision,Recall,F1
0,XGBoost,0.536769,0.990698,0.037467,0.072203


In [13]:
lgbm = lgb.LGBMClassifier(
    n_estimators=300,
    num_leaves=31,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    n_jobs=-1,
    random_state=42
)

lgbm.fit(X_train, y_train)

y_pred_lgb = lgbm.predict(X_test)
acc_lgb, prec_lgb, rec_lgb, f1_lgb = compute_metrics(y_test, y_pred_lgb, "LightGBM")

df_lgb = pd.DataFrame([[ "LightGBM", acc_lgb, prec_lgb, rec_lgb, f1_lgb ]],
                      columns=["Model","Accuracy","Precision","Recall","F1"])
df_lgb.to_csv("results/LightGBM.csv", index=False)
df_lgb


[LightGBM] [Info] Number of positive: 11870, number of negative: 43271
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.552818 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 377400
[LightGBM] [Info] Number of data points in the train set: 55141, number of used features: 1664
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.215266 -> initscore=-1.293468
[LightGBM] [Info] Start training from score -1.293468
🔍 LightGBM Metrics:
Accuracy : 0.5981213505965981
Precision: 0.9989339019189766
Recall   : 0.1648197009674582
F1-score : 0.2829533444058584


c:\Users\Admin\anaconda3\envs\ids\lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


,Model,Accuracy,Precision,Recall,F1
0,LightGBM,0.598121,0.998934,0.16482,0.282953


In [14]:
class WindowDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)
    def __len__(self):
        return len(self.X)
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

train_ds_mlp = WindowDataset(X_train, y_train)
val_ds_mlp   = WindowDataset(X_val, y_val)
test_ds_mlp  = WindowDataset(X_test, y_test)

train_loader_mlp = DataLoader(train_ds_mlp, batch_size=256, shuffle=True)
val_loader_mlp   = DataLoader(val_ds_mlp,   batch_size=256, shuffle=False)
test_loader_mlp  = DataLoader(test_ds_mlp,  batch_size=256, shuffle=False)


In [15]:
class MLP_IDS(nn.Module):
    def __init__(self, in_dim, hidden1=512, hidden2=256, num_classes=2):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_dim, hidden1),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(hidden1, hidden2),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(hidden2, num_classes)
        )
    def forward(self, x):
        return self.net(x)

mlp_model = MLP_IDS(in_dim=X_train.shape[1]).to(DEVICE)
class_weights_t = torch.tensor(class_weights_np, dtype=torch.float32, device=DEVICE)
criterion_mlp = nn.CrossEntropyLoss(weight=class_weights_t)
optimizer_mlp = torch.optim.Adam(mlp_model.parameters(), lr=1e-3)


In [16]:
EPOCHS_MLP = 10
best_val_acc_mlp = -1
best_model_mlp_path = "results/mlp_best.pth"

def acc_batch(logits, y):
    preds = logits.argmax(dim=-1)
    return (preds == y).float().mean().item()

for epoch in range(1, EPOCHS_MLP+1):
    mlp_model.train()
    tl, ta = 0, 0
    for xb, yb in train_loader_mlp:
        xb, yb = xb.to(DEVICE), yb.to(DEVICE)
        logits = mlp_model(xb)
        loss = criterion_mlp(logits, yb)
        optimizer_mlp.zero_grad()
        loss.backward()
        optimizer_mlp.step()
        tl += loss.item()
        ta += acc_batch(logits, yb)
    tl /= len(train_loader_mlp); ta /= len(train_loader_mlp)

    mlp_model.eval()
    vl, va = 0, 0
    with torch.no_grad():
        for xb, yb in val_loader_mlp:
            xb, yb = xb.to(DEVICE), yb.to(DEVICE)
            logits = mlp_model(xb)
            loss = criterion_mlp(logits, yb)
            vl += loss.item()
            va += acc_batch(logits, yb)
    vl /= len(val_loader_mlp); va /= len(val_loader_mlp)

    if va > best_val_acc_mlp:
        best_val_acc_mlp = va
        torch.save(mlp_model.state_dict(), best_model_mlp_path)
        print(f"💾 MLP best model saved at epoch {epoch} | Val Acc={va:.4f}")

    print(f"Epoch {epoch}/{EPOCHS_MLP} | Train Loss={tl:.4f} Acc={ta:.4f} | Val Loss={vl:.4f} Acc={va:.4f}")

print("MLP training done.")


💾 MLP best model saved at epoch 1 | Val Acc=0.8461
Epoch 1/10 | Train Loss=0.4160 Acc=0.8786 | Val Loss=0.7201 Acc=0.8461
Epoch 2/10 | Train Loss=0.3771 Acc=0.8870 | Val Loss=1.1650 Acc=0.7438
Epoch 3/10 | Train Loss=0.3473 Acc=0.8869 | Val Loss=0.8631 Acc=0.8099
Epoch 4/10 | Train Loss=0.3103 Acc=0.8877 | Val Loss=1.6025 Acc=0.6923
Epoch 5/10 | Train Loss=0.2711 Acc=0.9012 | Val Loss=0.9927 Acc=0.8243
Epoch 6/10 | Train Loss=0.2253 Acc=0.9177 | Val Loss=1.6839 Acc=0.7997
Epoch 7/10 | Train Loss=0.1899 Acc=0.9306 | Val Loss=1.5869 Acc=0.8115
Epoch 8/10 | Train Loss=0.1570 Acc=0.9439 | Val Loss=1.5945 Acc=0.8208
Epoch 9/10 | Train Loss=0.1341 Acc=0.9524 | Val Loss=2.2657 Acc=0.8002
Epoch 10/10 | Train Loss=0.1145 Acc=0.9588 | Val Loss=2.0525 Acc=0.8154
MLP training done.


In [17]:
mlp_model.load_state_dict(torch.load(best_model_mlp_path))
mlp_model.eval()

y_true_mlp, y_pred_mlp = [], []
with torch.no_grad():
    for xb, yb in test_loader_mlp:
        xb = xb.to(DEVICE)
        logits = mlp_model(xb)
        preds = logits.argmax(dim=-1)
        y_true_mlp.extend(yb.numpy().tolist())
        y_pred_mlp.extend(preds.cpu().numpy().tolist())

acc_mlp, prec_mlp, rec_mlp, f1_mlp = compute_metrics(y_true_mlp, y_pred_mlp, "MLP")

df_mlp = pd.DataFrame([[ "MLP", acc_mlp, prec_mlp, rec_mlp, f1_mlp ]],
                      columns=["Model","Accuracy","Precision","Recall","F1"])
df_mlp.to_csv("results/MLP.csv", index=False)
df_mlp


🔍 MLP Metrics:
Accuracy : 0.6026064144876027
Precision: 0.914501257334451
Recall   : 0.1919085312225154
F1-score : 0.3172433847048561


,Model,Accuracy,Precision,Recall,F1
0,MLP,0.602606,0.914501,0.191909,0.317243


In [18]:
# Mean-pool across the 32 timesteps -> (N, 52)
X_mean = X_seq.mean(axis=1)   # (N, 52)
print("X_mean shape:", X_mean.shape)

(X_train_m, y_train_m), (X_val_m, y_val_m), (X_test_m, y_test_m) = split_window_data(X_mean, y_window)

print("Train (mean):", X_train_m.shape, y_train_m.shape)
print("Val   (mean):", X_val_m.shape, y_val_m.shape)
print("Test  (mean):", X_test_m.shape, y_test_m.shape)

class LSTMDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)
    def __len__(self):
        return len(self.X)
    def __getitem__(self, idx):
        x = self.X[idx].unsqueeze(0)  # (1, 52) -> sequence length 1
        return x, self.y[idx]

train_loader_lstm = DataLoader(LSTMDataset(X_train_m, y_train_m), batch_size=256, shuffle=True)
val_loader_lstm   = DataLoader(LSTMDataset(X_val_m, y_val_m),   batch_size=256, shuffle=False)
test_loader_lstm  = DataLoader(LSTMDataset(X_test_m, y_test_m), batch_size=256, shuffle=False)


X_mean shape: (78773, 52)
Train (mean): (55141, 52) (55141,)
Val   (mean): (11815, 52) (11815,)
Test  (mean): (11817, 52) (11817,)


In [19]:
class LSTM_IDS(nn.Module):
    def __init__(self, feature_dim=52, hidden_dim=128, num_layers=1, num_classes=2):
        super().__init__()
        self.lstm = nn.LSTM(input_size=feature_dim,
                            hidden_size=hidden_dim,
                            num_layers=num_layers,
                            batch_first=True)
        self.fc = nn.Linear(hidden_dim, num_classes)

    def forward(self, x):
        # x: (B, 1, 52)
        out, _ = self.lstm(x)        # (B, 1, hidden_dim)
        last = out[:, -1, :]         # (B, hidden_dim)
        return self.fc(last)

lstm_model = LSTM_IDS(feature_dim=X_train_m.shape[1]).to(DEVICE)
criterion_lstm = nn.CrossEntropyLoss(weight=class_weights_t)
optimizer_lstm = torch.optim.Adam(lstm_model.parameters(), lr=1e-3)


In [20]:
EPOCHS_LSTM = 10
best_val_acc_lstm = -1
best_model_lstm_path = "results/lstm_best.pth"

for epoch in range(1, EPOCHS_LSTM+1):
    lstm_model.train()
    tl, ta = 0, 0
    for xb, yb in train_loader_lstm:
        xb, yb = xb.to(DEVICE), yb.to(DEVICE)
        logits = lstm_model(xb)
        loss = criterion_lstm(logits, yb)
        optimizer_lstm.zero_grad()
        loss.backward()
        optimizer_lstm.step()
        tl += loss.item()
        ta += acc_batch(logits, yb)
    tl /= len(train_loader_lstm); ta /= len(train_loader_lstm)

    lstm_model.eval()
    vl, va = 0, 0
    with torch.no_grad():
        for xb, yb in val_loader_lstm:
            xb, yb = xb.to(DEVICE), yb.to(DEVICE)
            logits = lstm_model(xb)
            loss = criterion_lstm(logits, yb)
            vl += loss.item()
            va += acc_batch(logits, yb)
    vl /= len(val_loader_lstm); va /= len(val_loader_lstm)

    if va > best_val_acc_lstm:
        best_val_acc_lstm = va
        torch.save(lstm_model.state_dict(), best_model_lstm_path)
        print(f"💾 LSTM best model saved at epoch {epoch} | Val Acc={va:.4f}")

    print(f"Epoch {epoch}/{EPOCHS_LSTM} | Train Loss={tl:.4f} Acc={ta:.4f} | Val Loss={vl:.4f} Acc={va:.4f}")

print("LSTM training done.")


💾 LSTM best model saved at epoch 1 | Val Acc=0.7952
Epoch 1/10 | Train Loss=0.4680 Acc=0.8599 | Val Loss=0.6721 Acc=0.7952
💾 LSTM best model saved at epoch 2 | Val Acc=0.8085
Epoch 2/10 | Train Loss=0.4040 Acc=0.8770 | Val Loss=0.7226 Acc=0.8085
Epoch 3/10 | Train Loss=0.3923 Acc=0.8834 | Val Loss=0.8138 Acc=0.7991
💾 LSTM best model saved at epoch 4 | Val Acc=0.8086
Epoch 4/10 | Train Loss=0.3841 Acc=0.8853 | Val Loss=0.8333 Acc=0.8086
Epoch 5/10 | Train Loss=0.3777 Acc=0.8864 | Val Loss=0.9105 Acc=0.8007
Epoch 6/10 | Train Loss=0.3715 Acc=0.8873 | Val Loss=0.9317 Acc=0.7998
Epoch 7/10 | Train Loss=0.3666 Acc=0.8870 | Val Loss=0.9275 Acc=0.8071
Epoch 8/10 | Train Loss=0.3626 Acc=0.8875 | Val Loss=1.0106 Acc=0.8017
Epoch 9/10 | Train Loss=0.3585 Acc=0.8881 | Val Loss=1.0109 Acc=0.8016
Epoch 10/10 | Train Loss=0.3552 Acc=0.8876 | Val Loss=1.0288 Acc=0.7935
LSTM training done.


In [21]:
lstm_model.load_state_dict(torch.load(best_model_lstm_path))
lstm_model.eval()

y_true_lstm, y_pred_lstm = [], []
with torch.no_grad():
    for xb, yb in test_loader_lstm:
        xb = xb.to(DEVICE)
        logits = lstm_model(xb)
        preds = logits.argmax(dim=-1)
        y_true_lstm.extend(yb.numpy().tolist())
        y_pred_lstm.extend(preds.cpu().numpy().tolist())

acc_lstm, prec_lstm, rec_lstm, f1_lstm = compute_metrics(y_true_lstm, y_pred_lstm, "LSTM")

df_lstm = pd.DataFrame([[ "LSTM", acc_lstm, prec_lstm, rec_lstm, f1_lstm ]],
                       columns=["Model","Accuracy","Precision","Recall","F1"])
df_lstm.to_csv("results/LSTM.csv", index=False)
df_lstm


🔍 LSTM Metrics:
Accuracy : 0.592959295929593
Precision: 0.7501429388221841
Recall   : 0.23078276165347406
F1-score : 0.35297282754909876


,Model,Accuracy,Precision,Recall,F1
0,LSTM,0.592959,0.750143,0.230783,0.352973


In [22]:
# Train only on NORMAL windows (y_train == 0)
X_train_norm = X_train[y_train == 0]

print("Normal windows for IsolationForest training:", X_train_norm.shape[0])

iso = IsolationForest(
    n_estimators=200,
    contamination=0.1,   # rough guess, can tune
    n_jobs=-1,
    random_state=42
)
iso.fit(X_train_norm)

# Predict: -1 = anomaly, 1 = normal
pred_if = iso.predict(X_test)
y_pred_if = (pred_if == -1).astype(int)  # 1 = attack

acc_if, prec_if, rec_if, f1_if = compute_metrics(y_test, y_pred_if, "IsolationForest")

df_if = pd.DataFrame([[ "IsolationForest", acc_if, prec_if, rec_if, f1_if ]],
                     columns=["Model","Accuracy","Precision","Recall","F1"])
df_if.to_csv("results/IsolationForest.csv", index=False)
df_if


Normal windows for IsolationForest training: 43271
🔍 IsolationForest Metrics:
Accuracy : 0.5335533553355336
Precision: 0.546430488459474
Recall   : 0.17906772207563765
F1-score : 0.269740328563858


,Model,Accuracy,Precision,Recall,F1
0,IsolationForest,0.533553,0.54643,0.179068,0.26974


In [23]:
class AE_IDS(nn.Module):
    def __init__(self, in_dim, hidden1=512, hidden2=256):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Linear(in_dim, hidden1),
            nn.ReLU(),
            nn.Linear(hidden1, hidden2),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.Linear(hidden2, hidden1),
            nn.ReLU(),
            nn.Linear(hidden1, in_dim)
        )
    def forward(self, x):
        z = self.encoder(x)
        recon = self.decoder(z)
        return recon

# Train AE on normal windows only (train split)
X_train_norm_ae = X_train[y_train == 0]
train_ds_ae = WindowDataset(X_train_norm_ae, np.zeros(len(X_train_norm_ae)))
train_loader_ae = DataLoader(train_ds_ae, batch_size=256, shuffle=True)

ae_model = AE_IDS(in_dim=X_train.shape[1]).to(DEVICE)
criterion_ae = nn.MSELoss()
optimizer_ae = torch.optim.Adam(ae_model.parameters(), lr=1e-3)


In [24]:
EPOCHS_AE = 10

for epoch in range(1, EPOCHS_AE+1):
    ae_model.train()
    total_loss = 0
    for xb, _ in train_loader_ae:
        xb = xb.to(DEVICE)
        recon = ae_model(xb)
        loss = criterion_ae(recon, xb)
        optimizer_ae.zero_grad()
        loss.backward()
        optimizer_ae.step()
        total_loss += loss.item()
    total_loss /= len(train_loader_ae)
    print(f"Epoch {epoch}/{EPOCHS_AE} | Recon Loss={total_loss:.6f}")

print("AE training done.")


Epoch 1/10 | Recon Loss=0.565026
Epoch 2/10 | Recon Loss=0.424239
Epoch 3/10 | Recon Loss=0.402436
Epoch 4/10 | Recon Loss=0.388899
Epoch 5/10 | Recon Loss=0.373739
Epoch 6/10 | Recon Loss=0.571275
Epoch 7/10 | Recon Loss=0.502089
Epoch 8/10 | Recon Loss=0.437249
Epoch 9/10 | Recon Loss=0.362847
Epoch 10/10 | Recon Loss=0.357585
AE training done.


In [25]:
ae_model.eval()

# Reconstruction error on train normals (for threshold)
with torch.no_grad():
    train_errs = []
    for xb, _ in train_loader_ae:
        xb = xb.to(DEVICE)
        recon = ae_model(xb)
        err = ((recon - xb)**2).mean(dim=1)
        train_errs.extend(err.cpu().numpy())
train_errs = np.array(train_errs)
threshold = np.percentile(train_errs, 95)  # top 5% as anomalies
print("AE threshold (95th percentile):", threshold)

# Apply to test set
test_ds_ae = WindowDataset(X_test, y_test)
test_loader_ae = DataLoader(test_ds_ae, batch_size=256, shuffle=False)

y_true_ae = []
y_pred_ae = []

with torch.no_grad():
    for xb, yb in test_loader_ae:
        xb = xb.to(DEVICE)
        recon = ae_model(xb)
        err = ((recon - xb)**2).mean(dim=1).cpu().numpy()
        preds = (err >= threshold).astype(int)  # 1 = attack
        y_true_ae.extend(yb.numpy().tolist())
        y_pred_ae.extend(preds.tolist())

acc_ae, prec_ae, rec_ae, f1_ae = compute_metrics(y_true_ae, y_pred_ae, "Autoencoder")

df_ae = pd.DataFrame([[ "Autoencoder", acc_ae, prec_ae, rec_ae, f1_ae ]],
                     columns=["Model","Accuracy","Precision","Recall","F1"])
df_ae.to_csv("results/Autoencoder.csv", index=False)
df_ae


AE threshold (95th percentile): 0.8186171
🔍 Autoencoder Metrics:
Accuracy : 0.6183464500296183
Precision: 0.8386167146974063
Recall   : 0.2559366754617414
F1-score : 0.3921832884097035


,Model,Accuracy,Precision,Recall,F1
0,Autoencoder,0.618346,0.838617,0.255937,0.392183


In [26]:
from sklearn.ensemble import RandomForestClassifier

print("\n🌲 Training Random Forest...")

rf = RandomForestClassifier(
    n_estimators=400,
    max_depth=25,
    min_samples_split=2,
    min_samples_leaf=1,
    n_jobs=-1,
    random_state=42
)

rf.fit(X_train, y_train)

y_pred_rf = rf.predict(X_test)

acc_rf, prec_rf, rec_rf, f1_rf = compute_metrics(y_test, y_pred_rf, "RandomForest")

df_rf = pd.DataFrame([[ "RandomForest", acc_rf, prec_rf, rec_rf, f1_rf ]],
                     columns=["Model","Accuracy","Precision","Recall","F1"])
df_rf.to_csv("results/RandomForest.csv", index=False)
df_rf



🌲 Training Random Forest...
🔍 RandomForest Metrics:
Accuracy : 0.51992891596852
Precision: 1.0
Recall   : 0.0021108179419525065
F1-score : 0.00421274354923644


,Model,Accuracy,Precision,Recall,F1
0,RandomForest,0.519929,1.0,0.002111,0.004213
